In [ ]:
%pip install --upgrade pip
%pip install openicl
# Restart the kernel after the installation is completed

# 3. Accelerating OpenICL with 🤗 Accelerate: Distributed Data Parallel and Model Parallel

In OpenICL, we use 🤗 [Accelerate](https://github.com/huggingface/accelerate) to implement Distributed Data Parallel (DDP) and Model Parallel. 🤗 [Accelerate](https://github.com/huggingface/accelerate) is a library that enables the same PyTorch code to be run across any distributed configuration by adding just few lines of code, to quickly quickly set up 🤗 Accelerate, on your machine(s) just run:

In [ ]:
accelerate config

For more details on 🤗 Accelearte, you can check the [documentation](https://huggingface.co/docs/accelerate/index) here.

## 3-1 Distributed Data Parallel

Distributed Data Parallel (DDP) implements data parallelism at the module level which can run across multiple machines. The recommended way to use DDP is to spawn one process for each model replica, where a model replica can span multiple devices. It is quite easy to use DDP in OpenICL after completing relevant settings through ```accelerate config```, just pass in the `Accelerator` instance in `Retriever` and `Inferencer`. The following are code and script examples:

In [ ]:
# test_sst2_ddp.py
# Example adapted from tutorial 1-4-1

from openicl import DatasetReader, PromptTemplate, TopkRetriever, PPLInferencer
from accelerate import Accelerator

# Accelerate Prepare
accelerator = Accelerator()

# Define a DatasetReader, loading dataset from huggingface.
data = DatasetReader('gpt3mix/sst2', input_columns=['text'], output_column='label')

# SST-2 Template Example
template = PromptTemplate(template={
                                        0: '</E>Positive Movie Review: </text>',
                                        1: '</E>Negative Movie Review: </text>' 
                                    },
                          column_token_map={'text' : '</text>'},
                          ice_token='</E>'
           )

# TopK Retriever
retriever = TopkRetriever(data, ice_num=8, index_split='train', test_split='test', accelerator=accelerator)

# Define a Inferencer
inferencer = PPLInferencer(model_name='distilgpt2', accelerator=accelerator)

# Inference
predictions = inferencer.inference(retriever, ice_template=template, output_json_filename='ddp_sst2')

# print(predictions)
# Seeing results at ./icl_inference_output/ddp_sst2.json

In [ ]:
# run_sst2.ddp.sh 
# Replace `${your_gpu_num}` and `${your_port_id}` with your gpu number and running port number respectively
accelerate launch --num_processes ${your_gpu_num} --main_process_port ${your_port_id} test_sst2_ddp.py

## 3-2 Model Parallel